In [4]:
import pandas as pd
import tweepy
import re
import nltk
import numpy as np
#from wordcloud import WordCloud, STOPWORDS

In [5]:
pd.set_option("display.max_colwidth", -1)

/var/folders/_0/21d2kw097h912yp8tr3nkrsm0000gn/T/ipykernel_20270/2475433900.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option("display.max_colwidth", -1)


### Twitter APIv2
- Authenticate with Twitter

In [6]:
#Authenticate with Twitter
client = tweepy.Client(bearer_token = 'SECRET')

In [ ]:
query = '#January6thCommitteeHearings lang:en'
paginator = tweepy.Paginator(
    client.search_recent_tweets,           # The method you want to use
    query=query,                           # Some argument for this method
    max_results=100,                       # How many tweets per page
    limit=350                               # How many pages to retrieve
)
# Get tweets that contain the hashtag #TypeKeywordHere
# -is:retweet means I don't want retweets
# lang:en is asking for the tweets to be in english

tweet_list = []

for tweet in paginator.flatten(): # Total number of tweets to retrieve
    tweet_list.append(tweet)
    #print(tweet)

### Create dataframe of tweets

In [8]:
tweet_list_df = pd.DataFrame(tweet_list)
tweet_list_df = pd.DataFrame(tweet_list_df['text'])
tweet_list_df.head(5)

,text
0,@ReallyAmerican1 #Roevember and\n#ForThePeople and\n#VoteBlueIn2022 and \n#StandWithUkraine️ and\n#PeopleOverPolitics and\n#OathBreakerMAGA and\n#LGBTQHistoryMonth and\n#InflationReductionAct and\n#RepublicanWarOnSeniors and\n#SocialSecurityIsOurMoney and\n#January6thCommitteeHearings...
1,RT @sandibachom: IS THIS THING ON???!!This is pathetic. ACTING Sec of Defense Chris Miller was the ONLY person who can deploy the national…
2,RT @sandibachom: IS THIS THING ON???!!This is pathetic. ACTING Sec of Defense Chris Miller was the ONLY person who can deploy the national…
3,RT @tleehumphrey: Today is the beginning of the inquiry into the Trudeau gov’s use of the Emergencies Act against the #FreedomConvoy so wha…
4,RT @AdamKinzinger: Mitch McConnell.\nKevin McCarthy.\nThey both knew Trump was responsible. \nThey both called him out.\nThey both backed down.…


### Data Preprocessing
- sub() function that returns a string after replacing the matched pattern in a string with a replacement.
- Syntax:
re.sub(pattern, repl, string, count=0, flags=0) https://www.pythontutorial.net/python-regex/python-regex-sub/

In [9]:
def preprocess_tweet(sen):
    '''Cleans text data up, leaving only 2 or more char long non-stopwords composed of A-Z & a-z only
    in lowercase'''
    
    sentence = sen.lower()

    # Remove RT
    sentence = re.sub('RT @\w+: '," ", sentence)

    # Remove special characters
    sentence = re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ", sentence)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)  

    # Remove multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)  

    return sentence

In [10]:
cleaned_tweets = []

for tweet in tweet_list_df['text']:
  cleaned_tweet = preprocess_tweet(tweet)
  cleaned_tweets.append(cleaned_tweet)

In [11]:
tweet_list_df['cleaned'] = pd.DataFrame(cleaned_tweets)

In [12]:
tweet_list_df.shape

(34993, 2)

In [13]:
tweet_list_df.head(50)

,text,cleaned
0,@ReallyAmerican1 #Roevember and\n#ForThePeople and\n#VoteBlueIn2022 and \n#StandWithUkraine️ and\n#PeopleOverPolitics and\n#OathBreakerMAGA and\n#LGBTQHistoryMonth and\n#InflationReductionAct and\n#RepublicanWarOnSeniors and\n#SocialSecurityIsOurMoney and\n#January6thCommitteeHearings...,roevember and forthepeople and votebluein2022 and standwithukraine and peopleoverpolitics and oathbreakermaga and lgbtqhistorymonth and inflationreductionact and republicanwaronseniors and socialsecurityisourmoney and january6thcommitteehearings
1,RT @sandibachom: IS THIS THING ON???!!This is pathetic. ACTING Sec of Defense Chris Miller was the ONLY person who can deploy the national…,rt is this thing on this is pathetic acting sec of defense chris miller was the only person who can deploy the national
2,RT @sandibachom: IS THIS THING ON???!!This is pathetic. ACTING Sec of Defense Chris Miller was the ONLY person who can deploy the national…,rt is this thing on this is pathetic acting sec of defense chris miller was the only person who can deploy the national
3,RT @tleehumphrey: Today is the beginning of the inquiry into the Trudeau gov’s use of the Emergencies Act against the #FreedomConvoy so wha…,rt today is the beginning of the inquiry into the trudeau gov use of the emergencies act against the freedomconvoy so wha
4,RT @AdamKinzinger: Mitch McConnell.\nKevin McCarthy.\nThey both knew Trump was responsible. \nThey both called him out.\nThey both backed down.…,rt mitch mcconnell kevin mccarthy they both knew trump was responsible they both called him out they both backed down
5,RT @libradunn: That shit was heavy. ✌🏽 y'all take care 🖤\n\n#January6thCommitteeHearings,rt that shit was heavy all take care january6thcommitteehearings
6,"RT @ImagesByFresh: Here’s another graphic designed by the talented team @ImagesByFresh. Go ahead, use it, share it and tweet the truth. Pl…",rt here another graphic designed by the talented team go ahead use it share it and tweet the truth pl
7,"RT @StandForBetter: 📺 NEW Video:\n\nTrump Lost, yet he decided to hatch a coup with the help of Mark Meadows, Roger Stone, Rudy Giuliani, and…",rt new video trump lost yet he decided to hatch coup with the help of mark meadows roger stone rudy giuliani and
8,RT @2RawTooReal: I still cant get over the fact that Nancy Pelosi is with the sh!ts 😳 she wanted all the smoke 💨 with Donald trump #January…,rt still cant get over the fact that nancy pelosi is with the sh ts she wanted all the smoke with donald trump january
9,RT @Resist_MAGA_GOP: Mark Hamill is correct. You can love this country without creating violence by overthrowing democracy. #January6thComm…,rt maga gop mark hamill is correct you can love this country without creating violence by overthrowing democracy january6thcomm


In [14]:
#tweet_list_df.to_csv('TweetBatch3.csv', index=False)